## Detect user intention in message

In [1]:
from utils import set_env

set_env("OPENAI_API_KEY")

In [20]:
# Intention schema
from typing import Literal
from pydantic import BaseModel, Field


class IntentionSchema(BaseModel):
    intention: Literal["chat", "record"] = Field(description="Intention detected from user message")

In [37]:
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, SystemMessage

llm = ChatOpenAI(model="gpt-4.1-nano", temperature=0.0)

with_structured = llm.with_structured_output(IntentionSchema)

system_prompt = """
You are an intention classifier. Analyze the user message and classify it as either "chat" or "record".

**Classification Rules:**
- `record`: Message describes an actual expense, purchase, or payment that happened (must include specific amounts or clear transaction statements like "I bought X for Y", "paid Z", "spent $X on Y")
- `chat`: Everything else, including questions about expenses, requests for information, conversations (even if they mention money-related topics)

**Important:** Messages can be in any language. Focus on detecting monetary references regardless of language.

**Output Format:** Return only valid JSON matching this schema:
```json
{
  "intention": "chat" | "record"
}
```

**Examples:**
- "buy milk for $20" → `record`
- "Pay netflix subscription for 120rb" → `record` 
- "I spent 50€ on groceries" → `record`
- "Berapa pengeluaran saya hari ini?" → `chat`
- "How much did I spend?" → `chat`
- "What's my budget?" → `chat`
- "How are you today?" → `chat`
- "Tell me a joke" → `chat`

Now classify following message:
"""

def detect_intention(message: str):
    messages = [
        SystemMessage(system_prompt),
        HumanMessage(message),
    ]
    result = with_structured.invoke(messages)
    print(f"Message: {message}, Intention: {result.intention.upper()}")

In [40]:
detect_intention("How are you?")
detect_intention("Bayar internet 200rb")
detect_intention("Apa kabar?")
detect_intention("Fresh milk $25")
detect_intention("Berapa pengeluaran saya hari ini?")
detect_intention("Ikan segar dari pasar 50k. Bayar parkir 5000")

# edge cases, we'll label this as record for now
detect_intention("Just bought apple for $5, why so expensive?")

Message: How are you?, Intention: CHAT
Message: Bayar internet 200rb, Intention: RECORD
Message: Apa kabar?, Intention: CHAT
Message: Fresh milk $25, Intention: RECORD
Message: Berapa pengeluaran saya hari ini?, Intention: CHAT
Message: Ikan segar dari pasar 50k. Bayar parkir 5000, Intention: RECORD
Message: Just bought apple for $5, why so expensive?, Intention: RECORD
